# Map matching
We have cleaned our data, now we can start our study on the taxi GPS trajectories. Our first step is to perform **map-matching** on the GPS trajectory data to get the corrected GPS coordinates and the road ID sequence for each trajectory.

### What is map matching
Map matching is the process of aligning raw GPS data points with a known road network to determine the most likely path traveled. Since GPS readings can be imprecise due to various factors, map-matching algorithms help improve accuracy by snapping these points to the road network. Techniques vary from geometric approaches to more advanced probabilistic models like Hidden Markov Models (HMM), which account for both the GPS noise and the logical connectivity of roads.

### Map matching with Valhalla's Meili
The map matching tool we'll be using is **Valhalla's Meili**. [Valhalla](https://valhalla.github.io/valhalla/) is an open source routing engine and accompanying libraries for use with OpenStreetMap data. [Meili](https://valhalla.github.io/valhalla/api/map-matching/api-reference/) is Valhalla's map matching service that aligns GPS traces to known roads or paths by comparing raw location data to OpenStreetMap's detailed road network. It can handle large-scale spatial data and is optimized for high-performance processing, making it ideal for applications like traffic analysis or location-based services.

### Setting up Valhalla on my machine
I mainly followed these two articles ([link1](https://ikespand.github.io/posts/meili/?source=post_page-----4ea6b1c88a4c--------------------------------), [link2](https://nickdoulos.com/posts/map-matching/)) to set up and run Valhalla server with docker on my Mac. 

# Open Street Map
[OpenStreetMap](https://www.openstreetmap.org/about) (abbreviated OSM) is a website that uses an open geographic database which is updated and maintained by a community of volunteers via open collaboration.

OSM uses unique identifiers called "way IDs" to label and reference specific road segments or pathways on the map. These way IDs are essential for identifying and retrieving individual map features. In our study, we can use the sequence of way IDs of the paths that a taxi passed through during the trip to uniquely describe the taxi trip. 

# Map matching demonstration
In this project, we'll use map matching for two purposes:
- get GPS trace that are snapped to the actual roads
- get the trajectory sequence (the sequence of way IDs) of each trip

Let's go through the process of map matching for the first taxi GPS trace with Valhalla's Meili for demonstration. 

In [63]:
import pandas as pd

# Read clean data file
df = pd.read_pickle('clean_data.pkl')
df.head()

,TRIP_ID,POLYLINE
10,1372636875620000233,"[[-8.619894,41.148009],[-8.620164,41.14773],[-..."
13,1372638595620000233,"[[-8.608716,41.153499],[-8.607627,41.153481],[..."
25,1372639092620000233,"[[-8.632737,41.168295],[-8.6328,41.16825],[-8...."
63,1372641600620000612,"[[-8.585811,41.148666],[-8.585757,41.148972],[..."
64,1372636896620000360,"[[-8.617599,41.146137],[-8.617581,41.14593],[-..."


In [64]:
df.iloc[0]['POLYLINE']

'[[-8.619894,41.148009],[-8.620164,41.14773],[-8.62065,41.148513],[-8.62092,41.150313],[-8.621208,41.151951],[-8.621118,41.153517],[-8.620884,41.155416],[-8.620938,41.155479],[-8.620974,41.155461],[-8.621028,41.155461],[-8.619777,41.155344],[-8.619282,41.155335],[-8.618112,41.155101],[-8.61534,41.154579],[-8.613297,41.153994],[-8.612064,41.153832],[-8.611911,41.155227],[-8.611794,41.156838],[-8.610804,41.157171],[-8.61021,41.15727],[-8.609508,41.157333],[-8.60949,41.157351]]'

In [65]:
# Get the first taxi GPS trace as a string
trace = df.iloc[0]['POLYLINE']

# Convert the string into a list of lists
trace = eval(trace)

# Convert it into a DataFrame with two columns
trace = pd.DataFrame(trace, columns=['lon', 'lat'])

# Change the column order
trace = trace[['lat', 'lon']]

In [66]:
trace.head()

,lat,lon
0,41.148009,-8.619894
1,41.147730,-8.620164
2,41.148513,-8.620650
3,41.150313,-8.620920
4,41.151951,-8.621208


### Send a request to Meili
From Valhalla's doc,   
"There are two separate Map Matching calls that perform different operations on an input set of latitude, longitude coordinates. The `trace_route` action returns the shape snapped to the road network and narrative directions, while `trace_attributes` returns detailed attribution along the portion of the route.  
It is important to note that all service requests should be **POST** because `shape` or `encoded_polyline` can be fairly large."

To get the sequence of way IDs, we'll use the `trace_attributes` action, and we'll prepare out POST request according to the format it requires.

In [67]:
# Prepare the JSON payload for POST requests for the trace_attribute action
# Convert the prepared DataFrame into a JSON
meili_coordinates = trace.to_json(orient='records')

# Providing needed data for the body of Meili's request
meili_head = '{"shape":'
# Parameters of the request
meili_tail = ""","search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""

# Combining all the string into a single request
meili_request_body = meili_head + meili_coordinates + meili_tail

In [69]:
# Send the request
import requests

# The URL of our local Valhalla server
# We're using the 'trace_attributes' action
url = "http://localhost:8002/trace_attributes"

# Providing headers to the request
headers = {'Content-type': 'application/json'}

# Sending a request
r = requests.post(url, data=str(meili_request_body), headers=headers)

In [70]:
import json

# Checking if the response from Meili was correct
if r.status_code == 200:
    # Parsing the JSON response
    response_text = json.loads(r.text)
else:
    print("Incorrect request, status code {}".format(r.status_code))

`response_text` contains all information returned by Meili, and the information we're interested in are: 
- the coordinates of the points snapped to the road, stored in the item `matched_points`
- the matched path, stored in item `shape`
- the sequence of way IDs of the trips, stored in the list of edges

In [71]:
# Get the list of matched points and convert it into a DataFrame
matched_trace = response_text['matched_points']
matched_trace = pd.DataFrame([(p['lat'], p['lon']) for p in matched_trace], columns=['lat_matched', 'lon_matched'])

# Combine it with the DataFrame of the original GPS trace
trace = pd.concat([trace, matched_trace], axis=1)
trace.head()

,lat,lon,lat_matched,lon_matched
0,41.148009,-8.619894,41.148011,-8.619900
1,41.147730,-8.620164,41.147719,-8.620153
2,41.148513,-8.620650,41.148513,-8.620646
3,41.150313,-8.620920,41.150301,-8.621013
4,41.151951,-8.621208,41.151921,-8.621124


From the documentation, "Valhalla routing, map-matching, and elevation services use an encoded polyline format to store a series of latitude, longitude coordinates as a single string. Polyline encoding greatly reduces the size of the route response or map-matching request, especially for longer routes or GPS traces."  

To get the matched path, we need to decode the polyline with the tool provided in the documentation.

In [72]:
# Get matched path by decoding the `shape` item
import decode

matched_path = response_text['shape']
matched_path = decode.decode(matched_path)
# Change the format to the required format in folium
matched_path = [[lat, lon] for lon, lat in matched_path]

# Get sequence of way ID
edges = response_text['edges']
seq = [e['way_id'] for e in edges]

# Remove only consecutive duplicates in the way ID sequence
# Because when the taxi is on the same road, we only count the way ID once
seq = pd.Series(seq)
seq = seq.loc[seq.shift() != seq]
seq = seq.tolist()

### Visualizion
Let's visualize the map-matching result. The original GPS trace is plot in red line and markers, the matched GPS trace in blue, and the matched path in green.  

Note that the matched GPS trace is a list of match results, and there is a one-to-one correspondence with the input set of latitude, longitude coordinates and this list of match results. The matched path is a series of latitude, longitude coordinates of the shape snapped to the road.

In [73]:
import folium

# Initalize the map
m = folium.Map(location=(trace.iloc[0]['lat'],trace.iloc[0]['lon']), zoom_start=15)

In [74]:
# Plot each GPS trace on different layers of the map
fg_raw = folium.FeatureGroup("raw GPS trace")
fg_matched = folium.FeatureGroup("matched GPS trace")
fg_matchedpath = folium.FeatureGroup("matched path")

# Plot the original GPS trace
folium.PolyLine(trace[['lat','lon']], color='red', opacity=0.5, weight=6).add_to(fg_raw)
# Add circle markers for each point on the GPS trace
for index, p in trace.iterrows():
    folium.CircleMarker(location=(p['lat'], p['lon']), 
                        radius=4, weight=0, fill_color='red', fill_opacity=1).add_to(fg_raw)

# Plot the matched GPS trace, and add circle markers for each point
folium.PolyLine(trace[['lat_matched','lon_matched']], color='blue', opacity=0.5,weight=6).add_to(fg_matched)
for index, p in trace.iterrows():
    folium.CircleMarker(location=(p['lat_matched'], p['lon_matched']), 
                        radius=4, weight=0, fill_color='blue', fill_opacity=1).add_to(fg_matched)

# Plot matched path
folium.PolyLine(matched_path, color='green', opacity=0.5,weight=5).add_to(fg_matchedpath)
for p in matched_path:
    folium.CircleMarker(location=p, 
                        radius=4, weight=0, fill_color='green', fill_opacity=1).add_to(fg_matchedpath)

# Add each layer to the map
fg_raw.add_to(m)
fg_matched.add_to(m)
fg_matchedpath.add_to(m)

# Add layer control to the map
folium.LayerControl(position='topright', collapsed=False).add_to(m)

In [75]:
# Show interactive map
m

We can see that the matched points (blue) corresponds one-to-one to the original points (red), while snapped to the road. The matched path (green) represents the accurate driving path of the taxi given by the map matching algorithm.

# Map matching for the dataset
In this project, we'll use map matching for two purposes:
- get GPS trace that are snapped to the actual roads
- get the trajectory sequence (the sequence of way IDs) of each trip

Following the same procedure shown in the above demonstration, we'll perform map matching for all taxi GPS traces in the dataset.

In [84]:
import pandas as pd
import numpy as np
import json
import requests

# Read clean data file
df = pd.read_pickle('clean_data.pkl')

In [85]:
def gen_request_data(coords_str):
    """
    From the original POLYLINE data, prepare the JSON payload for POST requests for the trace_attribute action of Meili.

    Args:
        coords_str (str): original POLYLINE data of the GPS trace

    Returns:
        str: JSON payload of the required format
    """
    # Convert the string into a list of lists
    coords_list = eval(coords_str)

    # Convert the list into desired JSON structure
    # Convert it into a DataFrame with two columns
    coords_df = pd.DataFrame(coords_list, columns=['lon', 'lat'])
    # Change the column order
    coords_df = coords_df[['lat', 'lon']]
    # Convert the DataFrame into a JSON
    meili_coordinates = coords_df.to_json(orient='records')

    # Providing needed data for the body of Meili's request
    meili_head = '{"shape":'
    # Parameters of the request
    meili_tail = ""","search_radius": 150, "shape_match":"map_snap", "costing":"auto", "format":"osrm"}"""

    # Combining all the string into a single request
    meili_request_body = meili_head + meili_coordinates + meili_tail

    return meili_request_body

In [86]:
def get_meili_response(meili_request_body_str):
    """
    Take the JSON payload, send a request to Meili, parse the JSON response if the response was correct.

    Args:
        meili_request_body_str (str): JSON payload for the POST request, generated by the function gen_request_data

    Raises:
        Exception: Raises an exception and print the request code and request message if the response from Meili was
        incorrect.

    Returns:
        str: response from the trace_attribute action
    """
    # The URL of our local Valhalla server
    # We're using the 'trace_attributes' action
    url = "http://localhost:8002/trace_attributes"

    # Providing headers to the request
    headers = {'Content-type': 'application/json'}

    # Send the request
    r = requests.post(url, data=meili_request_body_str, headers=headers)

    # Check if the response from Meili was correct
    if r.status_code == 200:

        # Parsing the JSON response
        response_text = json.loads(r.text)

        return response_text
    else:
        raise Exception("Request failed with status code {}: {}".format(r.status_code, r.text))

In [87]:
def matched_points(response_text):
    """
    Get the list of GPS coordinates of the matched points and convert it into a JSON format

    Args:
        response_text (str): response from the trace_attribute action

    Returns:
        str: list of matched points in JSON format
    """
    if response_text is None:
        return np.nan
        
    matched_trace = response_text['matched_points']

    if matched_trace:
        matched_trace_df = pd.DataFrame([(p['lat'], p['lon']) for p in matched_trace], columns=['lat', 'lon'])
        matched_trace_json = matched_trace_df.to_json(orient='records')
        
        return matched_trace_json
    else:
        return np.nan


def matched_path(response_text):
    """
    Get the matched path and convert it into a JSON format

    Args:
        response_text (str): response from the trace_attribute action

    Returns:
        str: matched path in JSON format
    """
    if response_text is None:
        return np.nan

    matched_path = response_text['shape']

    if matched_path:   
        # Get matched path by decoding the `shape` item
        import decode

        matched_path = decode.decode(matched_path)
        matched_path_df = pd.DataFrame(matched_path, columns=['lon', 'lat'])
        matched_path_df = matched_path_df[['lat', 'lon']]
        matched_path_json = matched_path_df.to_json(orient='records')

        return matched_path_json
    else:
        return np.nan


def wayid_seq(response_text):
    """
    Get the sequence of way ID

    Args:
        response_text (str): response from the trace_attribute action

    Returns:
        list: sequence of way ID, list of int
    """
    if response_text is None:
        return np.nan
        
    edges = response_text['edges']

    if edges:
        # Get sequence of way ID
        seq = [e['way_id'] for e in edges]
    
        # Remove only consecutive duplicates in the way ID sequence
        # Because when the taxi is on the same road, we only count the way ID once
        seq = pd.Series(seq)
        seq = seq.loc[seq.shift() != seq]
        seq = seq.tolist()
    
        return seq
    else:
        return np.nan

In [88]:
def get_wayid_seq(coords_str):
    """
    From the original POLYLINE data, make a request to Meili, get the sequence of way ID from the response.

    Args:
        coords_str (str): original POLYLINE data of the GPS trace

    Returns:
        list: sequence of way ID, list of int
    """
    meili_request_body_str = gen_request_data(coords_str)
    
    try:
        response_text = get_meili_response(meili_request_body_str)
    except Exception as e:
        print("Error: {}".format(e))  # Print the error
        response_text = None 
        
    seq = wayid_seq(response_text)

    return seq

In [89]:
df['wayid_seq'] = df['POLYLINE'].apply(get_wayid_seq)

In [91]:
# Save the result as .pkl file
df.to_pickle('map_matching_result.pkl')

In [92]:
df.head()

,TRIP_ID,POLYLINE,wayid_seq
10,1372636875620000233,"[[-8.619894,41.148009],[-8.620164,41.14773],[-...","[608421700, 1085592953, 1206667334, 595164143,..."
13,1372638595620000233,"[[-8.608716,41.153499],[-8.607627,41.153481],[...","[38336638, 210673563, 1010326126, 892703083, 8..."
25,1372639092620000233,"[[-8.632737,41.168295],[-8.6328,41.16825],[-8....",[232135018]
63,1372641600620000612,"[[-8.585811,41.148666],[-8.585757,41.148972],[...","[31721584, 549245879, 125150860, 859507142, 85..."
64,1372636896620000360,"[[-8.617599,41.146137],[-8.617581,41.14593],[-...","[14326493, 211436103, 697598025, 763524508, 11..."
